# Chains in LangChain

In [13]:
from L3 import *

In [14]:
import pandas as pd
df = pd.read_csv('Data.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


For linking single input-output chains

In [15]:
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)          

runnable = new_runnable(first_prompt, llm, output_key="company_name") \
    | new_runnable(second_prompt, llm, output_key="description") \
    | itemgetter("description")
    
invoke(runnable, product="AI generated Youtube videos")

'AI Tube Productions is a video production company that specializes in creating high-quality content using artificial intelligence technology.'

For linking linear feedforward chains

In [16]:
prompts = [
    ChatPromptTemplate.from_template(
        "Translate the following review to english:"
        "\n\n{Review}"
    ),
    ChatPromptTemplate.from_template(
        "Can you summarize the following review in 1 sentence:"
        "\n\n{English_Review}"
    ),
    ChatPromptTemplate.from_template(
        "What language is the following review:\n\n{Review}"
    ),
    ChatPromptTemplate.from_template(
        "Write a follow up response to the following "
        "summary in the specified language:"
        "\n\nSummary: {summary}\n\nLanguage: {language}"
    ),
]

output_keys = ["English_Review", "summary", "language", "followup_message"]
chains = [new_runnable(prompt, llm, output_key=key) for prompt, key in zip(prompts, output_keys)]

In [17]:
review = df.Review[5]
print(invoke_sequence(chains, Review=review))

{'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?", 'summary': 'The reviewer finds the taste of the product mediocre and suspects that it might be an old batch or counterfeit.', 'language': 'French.', 'followup_message': "Réponse de suivi:\n\nJe suis désolé d'apprendre que vous avez trouvé le goût du produit médiocre. Nous prenons la qualité de nos produits très au sérieux et nous sommes déterminés à fournir à nos clients des produits frais et authentiques. Nous aimerions en savoir plus sur votre expérience avec notre produit. Pourriez-vous nous fournir plus de détails sur la date d'expiration et le lieu d'achat du produit? Nous apprécions votre retour d'information et nous ferons tout notre possible pour résoudre ce problème."}


For routing the chains

In [18]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [19]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [20]:
route = new_route(llm, prompt_infos, verbose=True)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [21]:
print(invoke(route, input="What is black body radiation?"))
print(invoke(route, input="what is 2 + 2"))
print(invoke(route, input="Why does every cell in our body contain DNA?"))



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.
{'input': 'What is black body radiation?', 'text': "Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an object that absorbs all radiation that falls on it and emits radiation at all wavelengths. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding the behavior of stars, as well as in the development of technologies such as incandescent light bulbs and infrared cameras."}


> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.
{'input': 'what is 2 + 2', 'text': 'As an AI language model, I can answer this question easily. The answer to 2 + 2 is 4.'}


> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Fin